In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
from shapely.geometry import Point, Polygon

Vamos a 
1) Dataset con localidades urbanas. En ese dataset, incluye zonas urbanas agregadas
2) Cada localidad tiene el id del ageb
3) Hay como 170 mil localidades, en el shp hay 270. Las 100 mil 
O tienen todas las variables o solo poblacion
4) Imputar para cada ageb, si tenemos locaalidades con todos los datos, hagamos impute a localid


Una vez con eso listo, puedo hacer el merge entre las localidades y el shapefile
Hay algunas que no tendran match, 
las que no tienen poblacion es que no tienen poblacion
ahi no se imputa si solo tiene poblacion
para esas localidades, tomar el promedio de la ageb mas cercana
Si tienen una ageb vecinaa que tienen informacion, cogemos el valor del ageb vecino, o del promedio


he plan is still (1) for agebs with some missing microdata, input the average of the data in that ageb (2) for ageb people with no microdata, input the average from the neighboring agebs, if some agebs have no neighbors with any microdata let me know and we need to decide what to do with them (3) for agebs with no people, let’s keep the polygon but do not do any inputation

## HELPER FUNCTIONS

In [3]:
def convert_to_numeric(df, vars_to_numeric):
    '''
    Converts string values to numeric, making nulls those values with "N/D" or "*"
    Inputs:
            df: pandas dataframe
            vars_to_numeric: list of variable names
    Returns:
            Pandas dataframe
    '''
    df[vars_to_numeric] = df[vars_to_numeric].apply(pd.to_numeric, errors='coerce', axis = 1)

    return df


In [4]:
def add_vars_wealth_index(df):

    '''
    Computes the variables that will be used for constructing the wealth index
    Inputs:
        df- DataFrame
    Returns:
        Dataframe
    '''
    AS_PERCENT = 100

    # asset_ownsership_vars
    df['no_fridge'] = (df['VIVTOT'] - df['VPH_REFRI']) / df['VIVTOT'] * AS_PERCENT #1
    df['no_car'] = (df['VIVTOT'] - df['VPH_AUTOM']) / df['VIVTOT'] * AS_PERCENT #2
    df['no_internet'] = (df['VIVTOT'] - df['VPH_INTER']) / df['VIVTOT'] * AS_PERCENT #3
    df['no_washingmachine'] = (df['VIVTOT'] - df['VPH_LAVAD']) / df['VIVTOT'] * AS_PERCENT #4
    df['no_radio'] = (df['VIVTOT'] - df['VPH_RADIO']) / df['VIVTOT'] * AS_PERCENT #5
    df['no_tv'] = (df['VIVTOT'] - df['VPH_TV']) / df['VIVTOT'] * AS_PERCENT #6
    df['no_pc'] = (df['VIVTOT'] - df['VPH_PC']) / df['VIVTOT'] * AS_PERCENT #7
    df['no_cellphone'] = (df['VIVTOT'] - df['VPH_CEL']) / df['VIVTOT'] * AS_PERCENT #8
    df['no_telephone'] = (df['VIVTOT'] - df['VPH_TELEF']) / df['VIVTOT'] * AS_PERCENT #9
    return df


In [5]:
def build_query_for_nulls(list_of_vars, get_not_nulls=True):
    '''
    Creates a query to filter a data frame with multiple variables not containing nulls
    '''
    string = ""

    if get_not_nulls:
        for var in list_of_vars[:-1]:
            var = var + '.notnull() or ' 
            string += var
        string = string + list_of_vars[-1] + '.notnull()'
    else:
        for var in list_of_vars[:-1]:
            var = var + '.isnull() or ' 
            string += var
        string = string + list_of_vars[-1] + '.isnull()'
    return string

In [6]:
def split_data(df, list_of_vars):
    
    complete = df.query(build_query_for_nulls(ASSET_OWNERSHIP_vars,True), engine = 'python')
    incomplete = df.query(build_query_for_nulls(ASSET_OWNERSHIP_vars,False), engine = 'python')

    return complete, incomplete

In [7]:
def estimate_index(df):
    '''
    Estimates an assets index using PCA
    Inputs: 
        DataFrame
    Returns:
        Dataframe with two new columns for the principal component, and the index 0-1
    '''
    df = df.reset_index()
    X = convert_to_numeric(df, ASSET_OWNERSHIP_vars)
    X = df.loc[:, ASSET_OWNERSHIP_vars].to_numpy() # converts to numeric
    normalized_X = preprocessing.normalize(X) # normalizes the data
    pca = PCA(n_components = 1) # Create pca object
    pca = pca.fit_transform(normalized_X) # CCompute the first component
    first_component = pd.DataFrame(data = pca, columns = \
    ['pca'])
    min_pca =  pca.min() # Index 0-1
    max_pca = pca.max()
    difference = max_pca - min_pca
    
    first_component['index_pca'] = first_component['pca'].apply(lambda x: (x- min_pca)/difference)
    rv = df.join(first_component, lsuffix='_left', rsuffix='_right')
    rv = rv.set_index('index')
    return rv

## LOAD DATA AND ADD IDENTIFIERS

In [8]:
localidades_rurales = gpd.read_file('./Mexico_analysis/LOCALIDADES_RURALES.shp')

In [9]:
len(localidades_rurales)

289235

In [10]:
#File from https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=702825292805
localidades_rurales_2 = gpd.read_file('./shapefiles/rurales/mglr2010v5_0/localidades_rurales.shp')



In [11]:
len(localidades_rurales_2)

187722

In [12]:
# Shapefile provided by Alex
#Generate a code for a localidad in a state-municipality
localidades_rurales['localidad_code'] = localidades_rurales['CVE_ENT'] + \
localidades_rurales['CVE_MUN'] + localidades_rurales['CVE_LOC']

#Generate a code for a localidad in a state-municipality-AGEB

localidades_rurales['localidad_ageb'] = localidades_rurales['CVE_ENT'] + \
localidades_rurales['CVE_MUN'] + localidades_rurales['CVE_AGEB'] + localidades_rurales['CVE_LOC']

#Keep track of data source
localidades_rurales_2['source'] = 'Alex_shapefile'

In [13]:
#Shapefile downloaded from https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=702825292805

# First, let's make homogenous the AGEB clave. This shp has dashes
localidades_rurales_2['CVE_AGEB'] = localidades_rurales_2['CVE_AGEB'].apply(lambda x: x.replace("-",""))
#Generate a code for a localidad in a state-municipality
localidades_rurales_2 ['localidad_code'] = localidades_rurales_2['CVE_ENT'] + \
localidades_rurales_2['CVE_MUN'] + localidades_rurales_2['CVE_LOC']

#Generate a code for a localidad in a state-municipality-AGEB

localidades_rurales_2 ['localidad_ageb'] = localidades_rurales_2['CVE_ENT'] + \
localidades_rurales_2['CVE_MUN'] + localidades_rurales_2['CVE_AGEB']+localidades_rurales_2['CVE_LOC']

#Keep track of data source

localidades_rurales_2['source'] = 'mglr2010v5_0/'

In [14]:
# Create a list of localidades that are in the complement geodataframe 
#(localidades_rurales_2) which are not in the shapefile provided by Alex

different_localidades = set(list(localidades_rurales_2['localidad_ageb'])) - \
set(list(localidades_rurales['localidad_ageb'])) 

different_localidades = list(different_localidades)


In [21]:
print("There are ", len(different_localidades)," localidades that are not in the original data")

There are  6032  localidades that are not in the original data


In [22]:
#Let's filter the localidades that are not in the original data
complement_localidades = localidades_rurales_2[localidades_rurales_2['localidad_ageb'].isin (different_localidades)]

In [23]:
complement_localidades.head()

,CVE_ENT,CVE_MUN,CVE_AGEB,CVE_LOC,NOM_LOC,geometry,source,localidad_code,localidad_ageb
2261,07,017,0245,0591,El Recuerdo,POINT (3350518.942 526469.637),mglr2010v5_0/,070170591,0701702450591
6187,07,049,0018,0011,Kaomtealhucum,POINT (3480986.252 581101.780),mglr2010v5_0/,070490011,0704900180011
8315,07,059,0175,2030,El Retiro,POINT (3593711.705 590709.073),mglr2010v5_0/,070592030,0705901752030
18392,07,031,0136,0758,Bahtranca,POINT (3539393.520 601036.467),mglr2010v5_0/,070310758,0703101360758
19045,07,034,0087,0423,Los Tulipanes,POINT (3562282.943 442484.040),mglr2010v5_0/,070340423,0703400870423


In [24]:
# Now we have a complete dataset of localidades
all_localidades = pd.concat([pd.DataFrame(localidades_rurales),pd.DataFrame(complement_localidades)])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [25]:
all_localidades = all_localidades.sort_values("localidad_ageb") 

In [26]:
all_localidades.head()

,CVE_AGEB,CVE_ENT,CVE_LOC,CVE_MUN,NOM_LOC,geometry,localidad_ageb,localidad_code,source
212817,1068,01,0094,001,Granja Adelita,POINT (2461579.764 1095060.814),0100110680094,010010094,NaN
212830,1068,01,0127,001,Los Caños,POINT (2451892.325 1085142.936),0100110680127,010010127,NaN
212844,1068,01,0172,001,Granja San José,POINT (2460635.153 1094328.823),0100110680172,010010172,NaN
213546,1068,01,0421,001,Santa Cruz de la Presa,POINT (2461267.074 1095643.096),0100110680421,010010421,NaN
213547,1068,01,0422,001,Santa Cruz de la Presa (La Tlacuacha),POINT (2461089.151 1093164.684),0100110680422,010010422,NaN


In [27]:
print(len(localidades_rurales), len(all_localidades))

289235 295267


###  AGEBS Dataframe

In [28]:
agebs = gpd.read_file('./Mexico_analysis/AGEB.shp')

### Census data for localidades

In [29]:
all_states = pd.read_csv("./Mexico_analysis/all_states.csv", encoding = "utf8")
all_states['CVE_LOC'] = all_states['LOC'].apply(lambda x: str(x).zfill(4))
all_states['CVE_MUN'] = all_states['MUN'].apply(lambda x: str(x).zfill(3))
all_states['CVE_ENT'] = all_states['ENTIDAD'].apply(lambda x: str(x).zfill(2))

In [30]:
IDENTIFIERS = ['CVE_ENT','CVE_MUN','CVE_LOC','NOM_ENT', 'NOM_MUN', 'NOM_LOC', 'ENTIDAD', 'MUN', 'LOC', 'NOM_LOC.1',
       'LONGITUD', 'LATITUD']
POPULATION_vars= ['POBTOT', 'P15YM_AN', 'POB15_64','POB65_MAS', 'VIVTOT']
ASSET_OWNERSHIP_vars = [ 'VPH_RADIO', 'VPH_TV', 'VPH_REFRI','VPH_AUTOM',
    'VPH_INTER', 'VPH_LAVAD', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL']
asset_ownsership_vars = ['no_radio', 'no_tv','no_fridge', 'no_car',
    'no_internet','no_washingmachine', 'no_pc', 'no_telephone', 'no_cellphone']


In [31]:
all_states = all_states[IDENTIFIERS + POPULATION_vars + ASSET_OWNERSHIP_vars] # filters relevant data


In [32]:
all_states['localidad_code'] = all_states['CVE_ENT'] + all_states['CVE_MUN'] + all_states['CVE_LOC']

In [33]:
all_states = all_states.sort_values("localidad_code")

In [34]:
all_states = convert_to_numeric(all_states, POPULATION_vars + ASSET_OWNERSHIP_vars)

all_states = add_vars_wealth_index(all_states)

In [35]:
all_states.head()

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_ENT,NOM_MUN,NOM_LOC,ENTIDAD,MUN,LOC,NOM_LOC.1,...,localidad_code,no_fridge,no_car,no_internet,no_washingmachine,no_radio,no_tv,no_pc,no_cellphone,no_telephone
0,01,001,0001,Aguascalientes,Aguascalientes,Aguascalientes,1,1,1,Aguascalientes,...,010010001,23.015781,49.281492,75.589807,30.054410,25.495278,19.645140,65.336310,34.902010,53.713254
1,01,001,0094,Aguascalientes,Aguascalientes,Granja Adelita,1,1,94,Granja Adelita,...,010010094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,001,0096,Aguascalientes,Aguascalientes,Agua Azul,1,1,96,Agua Azul,...,010010096,28.571429,78.571429,92.857143,35.714286,42.857143,21.428571,92.857143,57.142857,100.000000
3,01,001,0100,Aguascalientes,Aguascalientes,Rancho Alegre,1,1,100,Rancho Alegre,...,010010100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,001,0102,Aguascalientes,Aguascalientes,Los Arbolitos [Rancho],1,1,102,Los Arbolitos [Rancho],...,010010102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
all_states[all_states['localidad_code']=='010012430']

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_ENT,NOM_MUN,NOM_LOC,ENTIDAD,MUN,LOC,NOM_LOC.1,...,localidad_code,no_fridge,no_car,no_internet,no_washingmachine,no_radio,no_tv,no_pc,no_cellphone,no_telephone
578,01,001,2430,Aguascalientes,Aguascalientes,Granjas del Valle,1,1,2430,Granjas del Valle,...,010012430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
all_localidades[all_localidades['localidad_code']=='010012430']

,CVE_AGEB,CVE_ENT,CVE_LOC,CVE_MUN,NOM_LOC,geometry,localidad_ageb,localidad_code,source
186288,1759,01,2430,001,Granjas del Valle,POINT (2463695.922 1096004.157),0100117592430,010012430,mglr2010v5_0/


In [ ]:
# localidades_rurales[(localidades_rurales['CVE_ENT']=="01")&
#                    (localidades_rurales['CVE_MUN']=="001")&
#                    (localidades_rurales['CVE_LOC']=="2430")]

In [ ]:
# localidades2[(localidades2['CVE_ENT']=="01")&
#                    (localidades2['CVE_MUN']=="001")&
#                    (localidades2['CVE_LOC']=="2430")]

## Let's split the data between those localidades which do have the numeric vars to construct the index, and those which do not have it

In [38]:
complete_data_states, incomplete_states = split_data(all_states, ASSET_OWNERSHIP_vars)

In [39]:
print(len(complete_data_states) + len(incomplete_states), len(all_states))

192247 192247


## Estimate index for rows that have complete information

In [55]:
index_complete = estimate_index(complete_data_states)

### Join with rows that contain missing data

In [56]:
states_with_index = pd.concat([index_complete, incomplete_states])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [57]:
states_with_index = states_with_index.sort_index()

In [58]:
states_with_index.columns

Index(['CVE_ENT', 'CVE_LOC', 'CVE_MUN', 'ENTIDAD', 'LATITUD', 'LOC',
       'LONGITUD', 'MUN', 'NOM_ENT', 'NOM_LOC', 'NOM_LOC.1', 'NOM_MUN',
       'P15YM_AN', 'POB15_64', 'POB65_MAS', 'POBTOT', 'VIVTOT', 'VPH_AUTOM',
       'VPH_CEL', 'VPH_INTER', 'VPH_LAVAD', 'VPH_PC', 'VPH_RADIO', 'VPH_REFRI',
       'VPH_TELEF', 'VPH_TV', 'index_pca', 'localidad_code', 'no_car',
       'no_cellphone', 'no_fridge', 'no_internet', 'no_pc', 'no_radio',
       'no_telephone', 'no_tv', 'no_washingmachine', 'pca'],
      dtype='object')

In [59]:
# Keep these columns
states_with_index = states_with_index.loc[:, ['CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'NOM_ENT', 'NOM_MUN', 'NOM_LOC',
       'ENTIDAD', 'MUN', 'LOC', 'localidad_code', 'NOM_LOC.1', 'LONGITUD', 'LATITUD', 'POBTOT',
       'P15YM_AN', 'POB15_64', 'POB65_MAS', 'VIVTOT','pca', 'index_pca']]



## MERGE with localidades shapefile to obtain the AGEBs' codes

In [60]:
all_localidades.head()

,CVE_AGEB,CVE_ENT,CVE_LOC,CVE_MUN,NOM_LOC,geometry,localidad_ageb,localidad_code,source
212817,1068,01,0094,001,Granja Adelita,POINT (2461579.764 1095060.814),0100110680094,010010094,NaN
212830,1068,01,0127,001,Los Caños,POINT (2451892.325 1085142.936),0100110680127,010010127,NaN
212844,1068,01,0172,001,Granja San José,POINT (2460635.153 1094328.823),0100110680172,010010172,NaN
213546,1068,01,0421,001,Santa Cruz de la Presa,POINT (2461267.074 1095643.096),0100110680421,010010421,NaN
213547,1068,01,0422,001,Santa Cruz de la Presa (La Tlacuacha),POINT (2461089.151 1093164.684),0100110680422,010010422,NaN


In [61]:
states_with_index.head()

,CVE_ENT,CVE_MUN,CVE_LOC,NOM_ENT,NOM_MUN,NOM_LOC,ENTIDAD,MUN,LOC,localidad_code,NOM_LOC.1,LONGITUD,LATITUD,POBTOT,P15YM_AN,POB15_64,POB65_MAS,VIVTOT,pca,index_pca
0,01,001,0001,Aguascalientes,Aguascalientes,Aguascalientes,1,1,1,010010001,Aguascalientes,1021746,215251,722250.0,11445.0,469897.0,37227.0,225328.0,-0.244952,0.244378
1,01,001,0094,Aguascalientes,Aguascalientes,Granja Adelita,1,1,94,010010094,Granja Adelita,1022225,215219,14.0,NaN,NaN,NaN,2.0,NaN,NaN
2,01,001,0096,Aguascalientes,Aguascalientes,Agua Azul,1,1,96,010010096,Agua Azul,1022127,215303,37.0,5.0,26.0,2.0,14.0,-0.266726,0.228629
3,01,001,0100,Aguascalientes,Aguascalientes,Rancho Alegre,1,1,100,010010100,Rancho Alegre,1022222,215116,10.0,NaN,NaN,NaN,1.0,NaN,NaN
4,01,001,0102,Aguascalientes,Aguascalientes,Los Arbolitos [Rancho],1,1,102,010010102,Los Arbolitos [Rancho],1022126,214649,7.0,NaN,NaN,NaN,2.0,NaN,NaN


In [47]:
agebs.head()

,CVE_ENT,CVE_MUN,CVE_AGEB,ID_ageb,geometry
0,30,061,0596,300610596,"POLYGON ((3330940.607 688385.876, 3331118.825 ..."
1,27,008,0154,270080154,"POLYGON ((3338816.762 688755.097, 3338835.358 ..."
2,02,001,6218,020016218,"POLYGON ((1128209.507 2311692.522, 1128260.522..."
3,02,003,0872,020030872,"POLYGON ((1127447.661 2314996.940, 1126963.004..."
4,27,007,0087,270070087,"POLYGON ((3589872.310 678498.925, 3589918.020 ..."


In [62]:
joined_data = states_with_index.set_index('localidad_code').join(all_localidades.set_index('localidad_code'), lsuffix='_left', rsuffix='_right').reset_index()

In [64]:
joined_data[joined_data['localidad_code']=='010012430']

,localidad_code,CVE_ENT_left,CVE_MUN_left,CVE_LOC_left,NOM_ENT,NOM_MUN,NOM_LOC_left,ENTIDAD,MUN,LOC,...,pca,index_pca,CVE_AGEB,CVE_ENT_right,CVE_LOC_right,CVE_MUN_right,NOM_LOC_right,geometry,localidad_ageb,source
578,010012430,01,001,2430,Aguascalientes,Aguascalientes,Granjas del Valle,1,1,2430,...,NaN,NaN,1759,01,2430,001,Granjas del Valle,POINT (2463695.922 1096004.157),0100117592430,mglr2010v5_0/


In [65]:
print("joined_data: ", len(joined_data), "obs", \
      "\nstates_with_index: ", len(states_with_index),"obs","\nAgeb: ", len(agebs), "obs")


joined_data:  192573 obs 
states_with_index:  192247 obs 
Ageb:  17442 obs


In [66]:
joined_data = joined_data.sort_values(['CVE_ENT_left', "CVE_AGEB"])

In [67]:
joined_data.head()

,localidad_code,CVE_ENT_left,CVE_MUN_left,CVE_LOC_left,NOM_ENT,NOM_MUN,NOM_LOC_left,ENTIDAD,MUN,LOC,...,pca,index_pca,CVE_AGEB,CVE_ENT_right,CVE_LOC_right,CVE_MUN_right,NOM_LOC_right,geometry,localidad_ageb,source
820,010030127,01,003,0127,Aguascalientes,Calvillo,La Hiedra,1,3,127,...,-0.197175,0.278935,0011,01,0127,003,La Hiedra,POINT (2426296.302 1111200.224),0100300110127,NaN
855,010030262,01,003,0262,Aguascalientes,Calvillo,Los Sabinos,1,3,262,...,NaN,NaN,0011,01,0262,003,Los Sabinos,POINT (2424785.880 1111575.083),0100300110262,NaN
1202,010060002,01,006,0002,Aguascalientes,PabellÃ³n de Arteaga,Las Ãnimas,1,6,2,...,-0.219021,0.263133,0013,01,0002,006,Las Ánimas,POINT (2472622.541 1119334.265),0100600130002,NaN
1203,010060003,01,006,0003,Aguascalientes,PabellÃ³n de Arteaga,Claravista [Granja],1,6,3,...,NaN,NaN,0013,01,0003,006,Granja Claravista,POINT (2470772.203 1121020.872),0100600130003,NaN
1204,010060005,01,006,0005,Aguascalientes,PabellÃ³n de Arteaga,Colonia GÃ¡mez Orozco (Puerta de Carboneras),1,6,5,...,-0.253633,0.238100,0013,01,0005,006,Colonia Gámez Orozco (Puerta de Carboneras),POINT (2474518.940 1127531.827),0100600130005,NaN


## IMPUTE MISSING VALUES OF LOCALIDADES TO AGEBS

### Create a key for each ageb within a municipality

In [68]:
joined_data['municipality_ageb'] = joined_data['CVE_ENT_left'] + joined_data['CVE_MUN_left'] + \
                        joined_data['CVE_AGEB'] 


### Impute

In [69]:
joined_data['index_imputed_to_Ageb'] = joined_data['index_pca'].fillna(joined_data.groupby('municipality_ageb')['index_pca'].transform('mean'))


In [70]:
joined_data[(joined_data['CVE_MUN_left']=='008')&
           (joined_data['CVE_ENT_left']=='01')][['localidad_code', 'CVE_LOC_left',
                                                'CVE_AGEB','municipality_ageb', 'index_pca',
                                                'index_imputed_to_Ageb']][:15]

,localidad_code,CVE_LOC_left,CVE_AGEB,municipality_ageb,index_pca,index_imputed_to_Ageb
1632,010080033,0033,0018,010080018,NaN,NaN
1648,010080180,0180,0022,010080022,NaN,NaN
1618,010080002,0002,0037,010080037,0.382139,0.382139
1619,010080007,0007,0037,010080037,0.379946,0.379946
1624,010080020,0020,0037,010080037,0.287810,0.287810
1630,010080029,0029,0037,010080037,0.275748,0.275748
1631,010080030,0030,0037,010080037,0.387620,0.387620
1634,010080061,0061,0037,010080037,0.260764,0.260764
1636,010080077,0077,0037,010080037,0.345059,0.345059
1640,010080117,0117,0037,010080037,NaN,0.332167


### This average is the same to the value of "index_imputed_to_Ageb" in rows 828 and 866

In [73]:
joined_data.isnull().sum()

localidad_code                0
CVE_ENT_left                  0
CVE_MUN_left                  0
CVE_LOC_left                  0
NOM_ENT                       0
NOM_MUN                       0
NOM_LOC_left                  0
ENTIDAD                       0
MUN                           0
LOC                           0
NOM_LOC.1                     0
LONGITUD                      0
LATITUD                       0
POBTOT                        0
P15YM_AN                  84825
POB15_64                  84825
POB65_MAS                 84825
VIVTOT                        0
pca                       84825
index_pca                 84825
CVE_AGEB                   4292
CVE_ENT_right              4292
CVE_LOC_right              4292
CVE_MUN_right              4292
NOM_LOC_right              4292
geometry                   4292
localidad_ageb             4292
source                   186541
municipality_ageb          4292
index_imputed_to_Ageb      5254
dtype: int64

In [83]:
joined_data[['CVE_ENT_left','CVE_MUN_left','CVE_LOC_left','CVE_AGEB', 'municipality_ageb',
             'localidad_code', 'index_pca', 'index_imputed_to_Ageb']][:20]

,CVE_ENT_left,CVE_MUN_left,CVE_LOC_left,CVE_AGEB,municipality_ageb,localidad_code,index_pca,index_imputed_to_Ageb
820,01,003,0127,0011,010030011,010030127,0.278935,0.278935
855,01,003,0262,0011,010030011,010030262,NaN,0.278935
1202,01,006,0002,0013,010060013,010060002,0.263133,0.263133
1203,01,006,0003,0013,010060013,010060003,NaN,0.271255
1204,01,006,0005,0013,010060013,010060005,0.238100,0.238100
1207,01,006,0014,0013,010060013,010060014,NaN,0.271255
1208,01,006,0015,0013,010060013,010060015,0.272150,0.272150
1209,01,006,0017,0013,010060013,010060017,NaN,0.271255
1210,01,006,0019,0013,010060013,010060019,0.271781,0.271781
1211,01,006,0021,0013,010060013,010060021,NaN,0.271255


In [91]:
joined_data[(joined_data['CVE_MUN_left']=='006')&
           (joined_data['CVE_ENT_left']=='01') &
           (joined_data['CVE_AGEB']=='0013')][[ 'index_pca']].mean()

index_pca    0.271255
dtype: float64

In [100]:
# Localidades with no index_pca nor imputed_index 

In [99]:
joined_data[(joined_data['index_imputed_to_Ageb'].isnull()) ]\
[['localidad_code', 'CVE_ENT_left', 'CVE_MUN_left', 'CVE_LOC_left',
       'NOM_ENT', 'POBTOT', 'NOM_LOC_left','CVE_AGEB','index_pca','index_imputed_to_Ageb']]



,localidad_code,CVE_ENT_left,CVE_MUN_left,CVE_LOC_left,NOM_ENT,POBTOT,NOM_LOC_left,CVE_AGEB,index_pca,index_imputed_to_Ageb
1632,010080033,01,008,0033,Aguascalientes,6.0,Antrialgo,0018,NaN,NaN
1648,010080180,01,008,0180,Aguascalientes,2.0,Puerta del MadroÃ±o,0022,NaN,NaN
1628,010080024,01,008,0024,Aguascalientes,1.0,Santa Rosa de Lima,0094,NaN,NaN
6371,020050057,02,005,0057,Baja California,4.0,La Providencia [Granja],0059,NaN,NaN
6394,020050094,02,005,0094,Baja California,6.0,Rancho Callado,0059,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
189986,320260345,32,026,0345,Zacatecas,9.0,Guadalupe,1490,NaN,NaN
189944,320260180,32,026,0180,Zacatecas,4.0,Palula,1537,NaN,NaN
189972,320260292,32,026,0292,Zacatecas,1.0,El Banco,1537,NaN,NaN
189985,320260341,32,026,0341,Zacatecas,3.0,DarÃ­a (Guadalupe),1537,NaN,NaN


## TO DO

In [ ]:
crs = {'init': 'epsg:4326'}

In [ ]:
gdf = gpd.GeoDataFrame(joined_data, crs=crs, geometry=joined_data['geometry'])

In [ ]:
type(gdf['geometry'][1])

In [ ]:
gdf[gdf['CVE_ENT_left']=="01"]['geometry'].plot()

In [ ]:
gpd.read_file("./shapefiles/32_Entidades_Federativas/LOCALIDADES_RURALES.shp")

In [ ]:
#gpd.read_file("./shapefiles/mglr2010v5_0/Localidades_rurales_2010_5.shp")

In [ ]:
from shapely.ops import nearest_points


In [ ]:
from covidmx import CovidMX

In [ ]:
covid_dge_data, catalogo_data, descripcion_data = CovidMX(return_catalogo=True, return_descripcion=True).get_data()


In [ ]:
covid_dge_data.columns

In [ ]:
covid_dge_data['resultado'].unique()

In [ ]:
len(covid_dge_data[covid_dge_data['resultado']=='Positivo SARS-CoV-2'])